# Condition Variables

In multithreaded programs, we often want to have one thread wait for some condition to be true before it can continue execution.

We might need one thread wait for some other thread to complete some action before it can continue execution.

We accomplish this using **condition variables**.

A **condition variable** is a queue of waiting threads.

We always need a lock associated with a condition variable. We have to grab the lock before using the condition variable.

Condition Variable Functions:

- `wait(cond *cv, lock *mutex)`
    - it is assumed that the thread calling wait holds the lock
    - adds thread to condition variable's queue, releases the lock, and puts the caller to sleep
    - When woken, wait will reacquire the lock and then return
- `signal(cond *cv)`
    - signals that one thread waiting on this condition variable can wake up
    - if no threads are waiting, it returns without doing anything


# Example: Implementing Join

```C
int done  = 0;
pthread_mutex_t m = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t c  = PTHREAD_COND_INITIALIZER;

void thr_exit() {
    Pthread_mutex_lock(&m);
    done = 1;
    Pthread_cond_signal(&c);
    Pthread_mutex_unlock(&m);
}

void *child(void *arg) {
    printf("child\n");
    thr_exit();
    return NULL;
}

void thr_join() {
    Pthread_mutex_lock(&m);
    while (done == 0)
        Pthread_cond_wait(&c, &m);
    Pthread_mutex_unlock(&m);
}

int main(int argc, char *argv[]) {
    printf("parent: begin\n");
    pthread_t p;
    Pthread_create(&p, NULL, child, NULL);
    thr_join();
    printf("parent: end\n");
    return 0;
}
```

Do we need the done variable?

```C

void thr_exit() {
    Pthread_mutex_lock(&m);
    Pthread_cond_signal(&c);
    Pthread_mutex_unlock(&m);
}

void thr_join() {
    Pthread_mutex_lock(&m);
    Pthread_cond_wait(&c, &m);
    Pthread_mutex_unlock(&m);
}
```

While using condition variables is a little complicated, there are some best practices which will make your life much easier:

1) always recheck the condition after calling on wait
   - the state of the condition might have changed between being woken and reaquiring the lock.
2) Always grab the lock before calling on `wait()` and release it ASAP after
3) Shouls always grab the lock before `signal()` and release it ASAP after

# The Producer/Consumer Model

The producer adds information to some shared buffer.

The consumer retrieves information from the shared buffer.

## Practical Example: A Web Server!

The producer accepts and adds web requests to a shared queue.

The consumers remove, process, and respond to the web requests.

### Condition Variables

Condition Variables are used to ensure that there is information in the buffer for a consumer to consume (otherwise consumers `wait()` for the buffer to hold information).

Condition variables are also used to ensure that the queue is not full.

If the buffer is empty, consumers will `wait()` until signaled by producers to continue.

If the buffer is full, producers will `wait()` until signaled by the consumers to continue.

## A second example: Shell Pipes

The pipe on the shell uses the producer/consumer model:

`cat test.txt | wc -l`

Pipe the output of `cat test.txt` into the input of `wc`

`wc` must wait until there is input in its buffer before it can start executing.

## Implementing Producer/Consumer

```c
int buffer;
int count = 0; //initially, empty

void put(int value) {
  assert(count == 0); // can only put a value into an empty buffer!
  count = 1; // now buffer will be full
  buffer = value;
}

void get() {
  assert(count == 1); // can only get a value from non-empty buffer!
  count = 0; // now buffer will be empty
  return buffer;
}
```

Our buffer holds a single value. Could expand it to be an array of values if we wanted.

The producer and consumer:

```c
// This will not work! The threads are not synchronized
int loops;

void *producer(void *args) {
  int i;
  for (i = 0; i < loops; i++) {
    put(i);
  }
}

void *consumer(void *args) {
  while(1){
    int tmp = get();
    printf("%d\n", tmp);
  }
}
```

## Synchronizing Threads

```c
// This will not work! The producer and consumers are all using the same cond variable
int loops;
cond_t cond;
lock_t mutex;

void *producer(void *args) {
  int i;
  for (i = 0; i < loops; i++) {
    Pthread_mutex_lock(&mutex);           // p1
    if(count == 1){                       // p2
      Pthread_cond_wait(&cond, &mutex);   // p3
    }
    put(i);                               // p4
    Pthread_cond_signal(&cond);           // p5
    Pthread_mutex_unlock(&mutex);         // p6
  }
}

void *consumer(void *args) {
  while(1){
    Pthread_mutex_lock(&mutex);           // c1
    if(count == 0){                       // c2
      Pthread_cond_wait(&cond, &mutex);   // c3
    }
    int tmp = get();                      // c4
    Pthread_cond_signal(&cond);           // c5
    Pthread_mutex_unlock(&mutex);         // c6
    printf("%d\n", tmp);
  }
}
```

Stepping through the code:

We have a problem because we are using a single condition variable for two separate conditions.

Suppose we have 1 producer and 2 consumers.

If both consumers run first:

Timeline:
```
                                            | C1 signals waking up C2
C1:  c1 c2 c3                         c4 c5 c6
C2:           c1 c2 c3                          c4 (fails, buffer empty)
P :                    p1 p2 p4 p5 p6
```

We need to use two condition variables since we have two conditions.

```c
// This will not work! The consumers need to recheck the condition before getting!
int loops;
cond_t full; // producer will signal that buffer is full
cond_t empty; // consumers will signal that buffer is empty
lock_t mutex;

void *producer(void *args) {
  int i;
  for (i = 0; i < loops; i++) {
    Pthread_mutex_lock(&mutex);           // p1
    if(count == 1){                       // p2
      Pthread_cond_wait(&empty, &mutex);  // p3
    }
    put(i);                               // p4
    Pthread_cond_signal(&full);           // p5
    Pthread_mutex_unlock(&mutex);         // p6
  }
}

void *consumer(void *args) {
  while(1){
    Pthread_mutex_lock(&mutex);           // c1
    if(count == 0){                       // c2
      Pthread_cond_wait(&full, &mutex);   // c3
    }
    int tmp = get();                      // c4
    Pthread_cond_signal(&empty);          // c5
    Pthread_mutex_unlock(&mutex);         // c6
    printf("%d\n", tmp);
  }
}
```

```
C1: c1 c2 c3                                                    c4 (fails no value!)
                                            C2 steals C1's value
C2:                                   c1 c2 c4 c5 c6 c1 c2 c3
P :          p1 p2 p4 p5 p6 p1 p2 p3 
```

```c
// This will not work! The consumers need to recheck the condition before getting!
int loops;
cond_t full; // producer will signal that buffer is full
cond_t empty; // consumers will signal that buffer is empty
lock_t mutex;

void *producer(void *args) {
  int i;
  for (i = 0; i < loops; i++) {
    Pthread_mutex_lock(&mutex);           // p1
    while(count == 1){                    // p2
      Pthread_cond_wait(&empty, &mutex);  // p3
    }
    put(i);                               // p4
    Pthread_cond_signal(&full);           // p5
    Pthread_mutex_unlock(&mutex);         // p6
  }
}

void *consumer(void *args) {
  while(1){
    Pthread_mutex_lock(&mutex);           // c1
    while(count == 0){                    // c2
      Pthread_cond_wait(&full, &mutex);   // c3
    }
    int tmp = get();                      // c4
    Pthread_cond_signal(&empty);          // c5
    Pthread_mutex_unlock(&mutex);         // c6
    printf("%d\n", tmp);
  }
}
```

## Condition Variables Best Practices

1) always recheck the condition after calling on wait
   - the state of the condition might have changed between being woken and reaquiring the lock.
2) Always grab the lock before calling on `wait()` and release it ASAP after
3) Shouls always grab the lock before `signal()` and release it ASAP after